# Facebook API - Fanpages

In [1]:
from facebook import GraphAPI
from requests import get
import simplejson as json
import pandas as pd
from tqdm import trange

In [2]:
from authData import URL

In [3]:
access_token = get(URL).json()['access_token']
del URL

In [4]:
api = GraphAPI(access_token, version='2.10')

In [5]:
class Data_formatting(object):    
    def print_json(item):
        print(json.dumps(item, indent=4, ensure_ascii=False, encoding='utf-8'))

class Fanpage(object):
    
    def __init__(self, fanpage_id):
        self.fanpage_id = fanpage_id

    def Posts(self, n_posts=25, limit=25, with_photo=False, next_post=None,):
        self.n_posts = n_posts
        self.limit = str(limit)
        self.batches = (n_posts//limit)
        self.with_photo = with_photo
        self.next_post = next_post
        
        if (self.with_photo == True) and (int(self.limit) > 100):
            from warnings import warn
            warn('The \'limit\' parameter should not exceed 100.', UserWarning)
            warn('\'with_photo\' parameter reseted to False. ', UserWarning)
            self.with_photo = False

        
        if self.next_post == None:
            if self.with_photo == True:
                r = '[{"method":"GET","relative_url":"'+self.fanpage_id+'/posts?limit='+self.limit+'"}, \
                             {"method":"GET","relative_url":"'+self.fanpage_id+'/posts?fields=full_picture'+ \
                                                                            '&limit='+self.limit+'"}]'
                raw_data = api.request("", post_args = {"batch": r})
                self.jsonPicture = json.loads(raw_data[1]['body'])
                self.jsonPost = json.loads(raw_data[0]['body'])
                paging_next = json.loads(raw_data[0]['body'])['paging']['next']
                
            else:
                raw_data = api.get_object(self.fanpage_id, fields='posts.limit('+self.limit+')')
                self.jsonPost = raw_data['posts']
                paging_next = raw_data['posts']['paging']['next']

                    
        self.dicReturn = {"data": []}
        
        
        if limit < 100:
            for i in trange(self.batches):
                self.next_post = get(paging_next).json()
                self.dicReturn['data'].extend(self.jsonPost['data'])
                paging_next = self.next_post['paging']['next']
            #add paging
            self.dicReturn.update({"paging":{'cursors':{'before': self.jsonPost['paging']['cursors']['before'],\
                                                     'after': self.jsonPost['paging']['cursors']['after']},\
                                                     'next': self.jsonPost['paging']['next']}})
        else:
            self.dicReturn['data'].extend(self.jsonPost['data'])
        
              
                
        if self.with_photo == True:        
            #add photo
            for i in self.jsonPicture['data']:
                for e in self.dicReturn['data']:
                    if i['id'] == e['id']:
                        self.dicReturn['data'][self.dicReturn['data'].index(e)].update(
                            {'full_picture': self.jsonPicture['data'][self.jsonPicture['data'].index(i)]['full_picture']})
        
        #Cheking if we have all the posts
        if len(self.dicReturn['data']) == self.n_posts: print("DONE! CHECK YOUR RESULTS!")
        
        return {self.fanpage_id: self.dicReturn}

In [6]:
teste = Fanpage('canseidesergato').Posts(n_posts=50, limit=50, with_photo=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.91it/s]


DONE! CHECK YOUR RESULTS!


In [7]:
Data_formatting.print_json(teste)

{
    "canseidesergato": {
        "data": [
            {
                "created_time": "2017-11-19T15:25:57+0000",
                "message": "quando você tá com fome e sente o cheiro do almoço do vizinho #canseidesergato",
                "id": "132428083633694_742413219301841",
                "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/s720x720/23658516_742413155968514_3823801765266480817_n.jpg?oh=65536ba2758cb062219e066af346d4d9&oe=5AAEA8EC"
            },
            {
                "created_time": "2017-11-18T18:20:00+0000",
                "message": "meu lugar preferido é deitado #canseidesergato",
                "id": "132428083633694_741961949346968",
                "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/s720x720/23561381_741961722680324_1024053445007012776_n.jpg?oh=f9e41c5f3e298e226819a1a612e7aea2&oe=5A8A8946"
            },
            {
                "created_time": "2017-11-17T17:56:04+0000",
                "message": "A MAIOR FAMÍL

}
